In [1]:
import matplotlib
matplotlib.use('qt5agg')
import matplotlib.pyplot as plt
import time
import numpy as np
import query_db
import composite
import matplotlib.gridspec as gridspec

In [2]:
def make_colorbar(composites):
    params = []
    # for comp in composites:
    # 	params.append(np.average(comp.dm15_array[comp.x1:comp.x2]))
    # norm = matplotlib.colors.Normalize(vmin=np.min(params),vmax=np.max(params))
    norm = matplotlib.colors.Normalize(vmin=1.,vmax=len(composites) + 1.)
    c_m = matplotlib.cm.plasma
    # c_m = matplotlib.cm.winter_r
    s_m = matplotlib.cm.ScalarMappable(cmap=c_m, norm=norm)
    s_m.set_array([])

    return s_m

In [27]:
def comparison_plot(composites):

#     plt.style.use('ggplot')
    colors = [color['color'] for color in list(plt.rcParams['axes.prop_cycle'])]
    h = [3,1,1,1,1,1,1]
    font = {'family' : 'serif',
        'color'  : 'black',
        'weight' : 'bold',
        'size'   : 10,
        }

    gs = gridspec.GridSpec(7, 1, height_ratios=h, hspace = .001)
    fig = plt.figure(num = 1, dpi = 100, figsize = [10,10])
    s_m = make_colorbar(composites)
    lw = 3

    composites, scales = composite.optimize_scales(composites, composites[0], True)

    i = 0
    k = 1
    for comp in composites:
        # param = np.average(comp.dm15_array[comp.x1:comp.x2])
        param = k

        rel_flux = plt.subplot(gs[0])
        plt.setp(rel_flux.get_xticklabels(), visible=False)
        plt.ylabel('Relative Flux', fontdict = font)
        rel_flux.axes.get_yaxis().set_ticks([])
        # plt.plot(comp.wavelength[comp.x1:comp.x2], comp.flux[comp.x1:comp.x2], color = s_m.to_rgba(param))
        plt.plot(comp.wavelength[comp.x1:comp.x2], comp.flux[comp.x1:comp.x2], linewidth = lw, color = s_m.to_rgba(param))
        if len(comp.low_conf) > 0 and len(comp.up_conf) > 0:
            plt.fill_between(comp.wavelength[comp.x1:comp.x2], comp.low_conf[comp.x1:comp.x2], 
                             comp.up_conf[comp.x1:comp.x2], color = s_m.to_rgba(param), alpha = 0.5)
            # plt.fill_between(comp.wavelength[comp.x1:comp.x2], comp.low_conf[comp.x1:comp.x2], 
            # 	             comp.up_conf[comp.x1:comp.x2], color = colors[i%len(colors)], alpha = 0.5)

        var = plt.subplot(gs[1], sharex = rel_flux)
        plt.setp(var.get_xticklabels(), visible=False)
        plt.ylabel('Variance', fontdict = font)
        # plt.plot(comp.wavelength[comp.x1:comp.x2], comp.ivar[comp.x1:comp.x2], color = s_m.to_rgba(param))
        plt.plot(comp.wavelength[comp.x1:comp.x2], comp.ivar[comp.x1:comp.x2], linewidth = lw, color = s_m.to_rgba(param))

        res = plt.subplot(gs[2], sharex = rel_flux)
        plt.setp(res.get_xticklabels(), visible=False)
        plt.ylabel('Residuals', fontdict = font)
        # plt.plot(comp.wavelength[comp.x1:comp.x2], comp.flux[comp.x1:comp.x2] - composites[0].flux[comp.x1:comp.x2], color = s_m.to_rgba(param))
        plt.plot(comp.wavelength[comp.x1:comp.x2], comp.flux[comp.x1:comp.x2] - composites[0].flux[comp.x1:comp.x2], linewidth = lw, color = s_m.to_rgba(param))
        if len(comp.low_conf) > 0 and len(comp.up_conf) > 0:
            low_resid = comp.low_conf[comp.x1:comp.x2] - composites[0].flux[comp.x1:comp.x2]
            up_resid = comp.up_conf[comp.x1:comp.x2] - composites[0].flux[comp.x1:comp.x2]
            plt.fill_between(comp.wavelength[comp.x1:comp.x2], low_resid, up_resid, color = s_m.to_rgba(param), alpha = 0.5)
            # plt.fill_between(comp.wavelength[comp.x1:comp.x2], low_resid, up_resid, color = colors[i%len(colors)], alpha = 0.5)

        spec = plt.subplot(gs[3], sharex = rel_flux)
        plt.setp(spec.get_xticklabels(), visible=False)
        plt.ylabel('Spectra/Bin', fontdict = font)
        # plt.plot(comp.wavelength[comp.x1:comp.x2], comp.spec_bin[comp.x1:comp.x2], color = s_m.to_rgba(param))
        plt.plot(comp.wavelength[comp.x1:comp.x2], comp.spec_bin[comp.x1:comp.x2], linewidth = lw, color = s_m.to_rgba(param))

        phase = plt.subplot(gs[4], sharex = rel_flux)
        plt.setp(phase.get_xticklabels(), visible=False)
        plt.ylabel('Phase [d]', fontdict = font)
        # plt.plot(comp.wavelength[comp.x1:comp.x2], comp.phase_array[comp.x1:comp.x2], color = s_m.to_rgba(param))
        plt.plot(comp.wavelength[comp.x1:comp.x2], comp.phase_array[comp.x1:comp.x2], linewidth = lw, color = s_m.to_rgba(param))

        delt = plt.subplot(gs[5], sharex = rel_flux)
        plt.setp(delt.get_xticklabels(), visible=False)
        plt.ylabel('$\Delta$m$_{15}$', fontdict = font)
        # plt.plot(comp.wavelength[comp.x1:comp.x2], comp.dm15[comp.x1:comp.x2], color = s_m.to_rgba(param))
        plt.plot(comp.wavelength[comp.x1:comp.x2], comp.dm15[comp.x1:comp.x2], linewidth = lw, color = s_m.to_rgba(param))

        plt.subplot(gs[6], sharex = rel_flux)
        plt.ylabel('Redshift', fontdict = font)
        # plt.plot(comp.wavelength[comp.x1:comp.x2], comp.red_array[comp.x1:comp.x2], color = s_m.to_rgba(param))
        plt.plot(comp.wavelength[comp.x1:comp.x2], comp.red_array[comp.x1:comp.x2], linewidth = lw, color = s_m.to_rgba(param))

        i+=1
        k+=1

    plt.xlabel('Wavelength ' + "($\mathrm{\AA}$)", fontdict = font)
    # cb = plt.colorbar(s_m, ax = fig.axes)
    # cb.set_label('Phase', fontdict = font)
    # for ax in fig.axes:
    # 	ax.set_axis_bgcolor('white')

    plt.show()

In [4]:
def stacked_plot_max(composites,buff,label_buff):
    font1 = {'family' : 'serif',
        'color'  : 'black',
        'weight' : 'bold',
        'size'   : 20,
        }
    
    font2 = {'family' : 'serif',
        'color'  : 'black',
        'weight' : 'bold',
        'size'   : 10,
        }
    
    fig, ax = plt.subplots(1,1)
    fig.set_size_inches(15, 30, forward = True)
    ax.get_yaxis().set_ticks([])
    ax.xaxis.set_tick_params(length=10, width = 2)
    plt.ylabel('Relative Flux + const.', fontdict = font1, fontsize = 40)
    plt.xlabel( 'Wavelength ' + "($\mathrm{\AA}$)", fontdict = font1, fontsize = 40)
    plt.xticks(fontsize = 20)
    composites, scales = composite.optimize_scales(composites, composites[0], True)

    i = 0
    for comp in composites:
        dm15 = np.average(comp.dm15[comp.x1:comp.x2])
#         ax.plot(comp.wavelength[comp.x1:comp.x2], 2.e15*comp.flux[comp.x1:comp.x2] - 1200*dm15, color = '#3F5D7D', linewidth = 4)
        ax.plot(comp.wavelength[comp.x1:comp.x2], 2.e15*comp.flux[comp.x1:comp.x2] - buff*i, color = '#3F5D7D', linewidth = 4)
        plt.fill_between(comp.wavelength[comp.x1:comp.x2], 2.e15*comp.low_conf[comp.x1:comp.x2]- buff*i, 2.e15*comp.up_conf[comp.x1:comp.x2] - buff*i, alpha = 0.5)
#         plt.text(9700, comp.flux[comp.x2] - 1200*dm15 + 30, '$\mathbf{\Delta m_{15}}$ = ' + str(round(dm15, 2)), fontdict = font1)
        plt.text(8000, comp.flux[comp.x2] - buff*i +label_buff, '$\mathbf{\Delta m_{15}}$ = ' + str(round(dm15, 2)), fontdict = font2, fontsize = 30)
        i += 1
#     plt.title('Maximum Light Composite Spectra \n for Different Light Curve Shapes', fontdict = font1, fontsize = 40)
    plt.xlim([3000,10500])
#     plt.savefig('../../FOE_figures/dm15_split_max.png', dpi = 300, bbox_inches = 'tight')
    plt.show()

# Max-No dm15 Match

In [5]:
query_host_max = ["SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= -3 and phase <= 3 and morphology >= 9",
                  "SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= -3 and phase <= 3 and morphology >= 5 and morphology <= 8",
                  "SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= -3 and phase <= 3 and morphology <= 4"]

In [6]:
comps_host_max = query_db.main(len(query_host_max), query_host_max)

SQL Query: SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= -3 and phase <= 3 and morphology >= 9
33 spectra found
151 questionable files currently ignored
bad file removed
bad file removed
2 questionable spectra removed 31 spectra left
2 Peculiar Ias removed 29 spectra left
27 SNs with host corrections
Arrays cleaned
151 questionable files currently ignored
1 spectra with nan ivars removed 26 spectra left
1999aa sn1999aa-19990223-ui.flm 0.24 0.78 None
Host correction...
1999ac sn1999ac-19990312-ui-corrected.flm -0.89 1.23 None
Host correction...
1999dq sn1999dq-near-19990917-clear-ui-corrected.flm 2.97 0.86 None
Host correction...
2002de sn2002de-20020608.374-ui-corrected.flm -0.32 0.96 None
Host correction...
2003u sn2003U-20030204-ui.flm -2.55 1.46 None
Host correction...
2004fu sn2004fu-20041119-ui.flm 2.43 1.08 None
Host correction...
2005eq sn2005eq-20051011.376-ui-corrected.flm 0.66 0.88 None
Host correction...
2005lz sn2005lz-200601

In [7]:
query_db.set_min_num_spec(comps_host_max, 5)
comparison_plot(comps_host_max)
stacked_plot_max(comps_host_max,5,1)

# Max-Low dm15 Matched

In [8]:
query_host_low_dm15 = ["SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= -3 and phase <= 3 and morphology >= 6 and ((dm15_cfa < 1.1 or dm15_from_fits < 1.1))",
                       "SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= -3 and phase <= 3 and morphology <= 5 and ((dm15_cfa < 1.15 or dm15_from_fits < 1.15))"]

In [9]:
comps_host_low_dm15 = query_db.main(len(query_host_low_dm15), query_host_low_dm15)

SQL Query: SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= -3 and phase <= 3 and morphology >= 6 and ((dm15_cfa < 1.1 or dm15_from_fits < 1.1))
48 spectra found
151 questionable files currently ignored
bad file removed
bad file removed
bad file removed
bad file removed
bad file removed
5 questionable spectra removed 43 spectra left
0 Peculiar Ias removed 43 spectra left
39 SNs with host corrections
Arrays cleaned
151 questionable files currently ignored
1 spectra with nan ivars removed 38 spectra left
1994s sn1994s-19940616-uoi.flm 1.11 0.94 None
Host correction...
1996ai sn1996ai-19960622-uoi.flm 1.231 0.88 None
Host correction...
1998dk sn1998dk-19980831-ui.flm -0.54 1.05 None
Host correction...
1999aa sn1999aa-19990223-ui.flm 0.24 0.78 None
Host correction...
1999dq sn1999dq-near-19990917-clear-ui-corrected.flm 2.97 0.86 None
Host correction...
2001cp sn2001cp-20010629-ui.flm 1.39 0.88 None
Host correction...
2002cd sn2002cd-20020420-ui

In [25]:
query_db.set_min_num_spec(comps_host_low_dm15, 5)
comparison_plot(comps_host_low_dm15)
stacked_plot_max(comps_host_low_dm15,5,1)

# Max-High dm15 Matched

In [11]:
query_host_high_dm15 = ["SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= -3 and phase <= 3 and morphology >= 6 and ((dm15_cfa > 1.3 or dm15_from_fits > 1.3)) and ((dm15_cfa < 1.9 or dm15_from_fits > 1.9))",
                        "SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= -3 and phase <= 3 and morphology <= 5 and ((dm15_cfa > 1.2 or dm15_from_fits > 1.2)) and ((dm15_cfa < 1.5 or dm15_from_fits < 1.5))"]

In [12]:
comps_host_high_dm15 = query_db.main(len(query_host_high_dm15), query_host_high_dm15)

SQL Query: SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= -3 and phase <= 3 and morphology >= 6 and ((dm15_cfa > 1.3 or dm15_from_fits > 1.3)) and ((dm15_cfa < 1.9 or dm15_from_fits > 1.9))
16 spectra found
151 questionable files currently ignored
bad file removed
bad file removed
bad file removed
3 questionable spectra removed 13 spectra left
2 Peculiar Ias removed 11 spectra left
8 SNs with host corrections
Arrays cleaned
151 questionable files currently ignored
0 spectra with nan ivars removed 8 spectra left
2003u sn2003U-20030204-ui.flm -2.55 1.46 None
Host correction...
2005lz sn2005lz-20060101.381-deimos.flm 0.58 1.34 None
Host correction...
1998dx sn1998dx-19980914.18-fast.flm -1.72 1.32 None
Host correction...
1999cc sn1999cc-19990516.34-fast.flm -1.26 1.35 None
Host correction...
2003it sn2003it-20031020.23-fast.flm -2.27 1.36 None
Host correction...
2004ef SN04ef_040914_b01_DUP_WF.dat -1.35000000009 1.39 None
Host correction...


In [14]:
query_db.set_min_num_spec(comps_host_high_dm15, 5)
comparison_plot(comps_host_high_dm15)
stacked_plot_max(comps_host_high_dm15,5,1)

# +1 Week-Low dm15 Matched

In [15]:
query_host_low_dm15_p1week = ["SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= 5 and phase <= 9 and morphology >= 6 and ((dm15_cfa < 1.1 or dm15_from_fits < 1.1))",
                              "SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= 5 and phase <= 9 and morphology <= 5 and ((dm15_cfa < 1.15 or dm15_from_fits < 1.15))"]

In [16]:
comps_host_low_dm15_p1week = query_db.main(len(query_host_low_dm15_p1week), query_host_low_dm15_p1week)

SQL Query: SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= 5 and phase <= 9 and morphology >= 6 and ((dm15_cfa < 1.1 or dm15_from_fits < 1.1))
41 spectra found
151 questionable files currently ignored
0 questionable spectra removed 41 spectra left
0 Peculiar Ias removed 41 spectra left
36 SNs with host corrections
Arrays cleaned
151 questionable files currently ignored
0 spectra with nan ivars removed 36 spectra left
1997bp sn1997bp-19970416-uoi.flm 5.49 1.08 None
Host correction...
1998v sn1998v-19980326-br.flm 7.2 0.99 None
Host correction...
2000fa sn2000fa-20001221-ui-corrected.flm 6.86 1.0 None
Host correction...
2001ay sn2001ay-20010430-ui.flm 6.79 0.68 None
Host correction...
2002de sn2002de-20020617-ui-corrected.flm 8.37 0.96 None
Host correction...
2005m sn2005m-20050211.413-br.flm 8.23 0.87 None
Host correction...
2006cm sn2006cm-20060605.470-ui.flm 6.77 1.04 None
Host correction...
2007le sn2007le-20071102.182-ui-corrected.flm 7

In [23]:
query_db.set_min_num_spec(comps_host_low_dm15_p1week, 5)
comparison_plot(comps_host_low_dm15_p1week)
stacked_plot_max(comps_host_low_dm15_p1week,5,1)

# +1 Week-High dm15 Matched

In [18]:
query_host_high_dm15_p1week = ["SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= 5 and phase <= 9 and morphology >= 6 and ((dm15_cfa > 1.3 or dm15_from_fits > 1.2)) and ((dm15_cfa < 1.7 or dm15_from_fits > 1.7))",
                              "SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= 5 and phase <= 9 and morphology <= 5 and ((dm15_cfa > 1.3 or dm15_from_fits > 1.3)) and ((dm15_cfa < 1.6 or dm15_from_fits > 1.6))"]

In [19]:
comps_host_high_dm15_p1week = query_db.main(len(query_host_high_dm15_p1week), query_host_high_dm15_p1week)

SQL Query: SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= 5 and phase <= 9 and morphology >= 6 and ((dm15_cfa > 1.3 or dm15_from_fits > 1.2)) and ((dm15_cfa < 1.7 or dm15_from_fits > 1.7))
14 spectra found
151 questionable files currently ignored
bad file removed
1 questionable spectra removed 13 spectra left
0 Peculiar Ias removed 13 spectra left
8 SNs with host corrections
Arrays cleaned
151 questionable files currently ignored
0 spectra with nan ivars removed 8 spectra left
1998dx sn1998dx-19980921-ui.flm 5.13 1.32 None
Host correction...
2001fh sn2001fh-20011116-ui-corrected.flm 5.93 1.51 None
Host correction...
2003iv sn2003iv-20031028-ui.flm 6.58 1.65 None
Host correction...
2004ef sn2004ef-20040924.302-ui-corrected.flm 8.05 1.39 None
Host correction...
2000cn sn2000cn-20000621.37-fast.flm 8.47 1.51 None
Host correction...
2006ev sn2006ev-20060916.23-fast.flm 5.63 1.32 None
Host correction...
2006gj SN06gj_060927_b01_DUP_WF.dat 6.18

In [24]:
query_db.set_min_num_spec(comps_host_high_dm15_p1week, 5)
comparison_plot(comps_host_high_dm15_p1week)
stacked_plot_max(comps_host_high_dm15_p1week,5,1)